# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [25]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters="enabled"
%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
base_path = 'D:/Steven/Facultad/MAESTRIA/DMEyF/segmentacion/'
dataset_path = base_path + 'datos/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_clase_ternaria.csv'

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

In [41]:
%%sql
CREATE OR REPLACE MACRO ratio_seguro(n, d) AS (
  CASE
    WHEN d IS NULL OR d = 0 THEN NULL
    ELSE CAST(n AS DOUBLE) / CAST(d AS DOUBLE)
  END
);

-- Porcentaje seguro (0–100)
CREATE OR REPLACE MACRO ratio_pct_seguro(n, d) AS (
  CASE
    WHEN d IS NULL OR d = 0 THEN NULL
    ELSE 100.0 * CAST(n AS DOUBLE) / CAST(d AS DOUBLE)
  END
);

-- Helper: división segura (reutilizable)
CREATE OR REPLACE MACRO div_segura(n, d) AS (
  CASE
    WHEN d IS NULL OR d = 0 THEN NULL
    ELSE CAST(n AS DOUBLE) / CAST(d AS DOUBLE)
  END
);

-- log1p(segura) con epsilon para evitar -1 por rounding
-- Si el ratio <= -1 + 1e-12 -> NULL (evita log(0) y nums negativos)
CREATE OR REPLACE MACRO ratio_log1p_seguro(n, d) AS (
  CASE
    WHEN n IS NULL OR d IS NULL OR d = 0 THEN NULL
    WHEN div_segura(n, d) IS NULL THEN NULL
    WHEN div_segura(n, d) <= (-1.0 + 1e-12) THEN NULL
    ELSE log10(div_segura(n, d))
  END
);


,Success


In [44]:
%%sql
ROLLBACK; -- por si acaso

,Success


In [45]:
%%sql
CREATE TABLE dataset_202104_con_ratios AS
SELECT
  t.*,

  -- ---- Uso de crédito
  ratio_pct_seguro(
    COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0),
    NULLIF(COALESCE(Visa_mlimitecompra, 0) + COALESCE(Master_mlimitecompra, 0), 0)
  )                                                   AS pct_util_limite_tc,

  ratio_pct_seguro(Visa_mpagominimo, NULLIF(Visa_msaldopesos, 0))
                                                    AS pct_pago_min_vs_saldo_visa,

  ratio_seguro(
    COALESCE(mtarjeta_visa_consumo, 0) + COALESCE(mtarjeta_master_consumo, 0),
    mcuentas_saldo
  )                                                   AS gasto_tc_vs_saldo,

  -- ---- Liquidez / sueldo
  ratio_seguro(mcaja_ahorro, mpayroll)               AS caja_vs_sueldo,
  ratio_seguro(mcuentas_saldo, mpayroll)             AS saldo_vs_sueldo,

  -- ---- Actividad transaccional
  ratio_seguro(ctarjeta_debito_transacciones, ctrx_quarter)
                                                     AS share_trx_debito,
  ratio_seguro(
    COALESCE(ctarjeta_visa_transacciones, 0) + COALESCE(ctarjeta_master_transacciones, 0),
    ctrx_quarter
  )                                                   AS share_trx_credito,
  ratio_seguro(
    COALESCE(chomebanking_transacciones, 0) + COALESCE(cmobile_app_trx, 0),
    ctrx_quarter
  )                                                   AS share_trx_digital,

  -- ---- Transferencias
  ratio_seguro(mtransferencias_emitidas, mtransferencias_recibidas)
                                                     AS ratio_out_in_monto,
  ratio_seguro(ctransferencias_emitidas, ctrx_quarter)
                                                     AS share_trx_out,

  -- ---- Estructura financiera
  ratio_seguro(mpasivos_margen, mactivos_margen)     AS pasivos_sobre_activos_margen,
  ratio_seguro(mcuenta_corriente, mcuentas_saldo)    AS ccorriente_vs_saldo,



FROM competencia_01 AS t
WHERE foto_mes = 202104;

,Success


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [ ]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.CatalogException) Catalog Error: Table with name dataset_enriquecido does not exist!
Did you mean "pg_sequence"?

LINE 34: FROM dataset_enriquecido
              ^
[SQL: CREATE OR REPLACE VIEW v_ratios_202104 AS
SELECT
  numero_de_cliente,
  foto_mes,


  ratio_pct_seguro(Visa_msaldototal + Master_msaldototal,
                   NULLIF(Visa_mlimitecompra,0) + NULLIF(Master_mlimitecompra,0)) AS pct_util_limite_tc,
  ratio_pct_seguro(Visa_mpagominimo, NULLIF(Visa_msaldopesos,0))                 AS pct_pago_min_vs_saldo_visa,
  ratio_seguro(mtarjeta_visa_consumo + mtarjeta_master_consumo, mcuentas_saldo)  AS gasto_tc_vs_saldo,


  ratio_seguro(mcaja_ahorro, mpayroll)                                           AS caja_vs_sueldo,
  ratio_seguro(mcuentas_saldo, mpayr

Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [ ]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [ ]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [47]:
%%sql
COPY dataset_202104_con_ratios TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);


,Success
